In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import glob
import cv2
from albumentations import Rotate
from scipy import ndimage
import imutils
from skimage import data
from skimage import color
from skimage.filters import meijering, sato, frangi, hessian
from skimage.morphology import medial_axis, skeletonize, thin, remove_small_objects
import rdp
import math
from tqdm import tqdm
import sys
sys.path.append('../Scripts/')
from segmentation_utils import *
from metrics import *
from image_utils import *

In [2]:
def calculate(path_list):
    df = pd.DataFrame(columns = ['img_id','tort', 'dm', 'ip', 'tp', 'sd'])
    all_metrics = []
    all_img_ids = []
    for i, path in tqdm(enumerate(path_list)):
        img_orig = cv2.imread(path, 0)
        seg_img = segment(img_orig)
        all_points = get_points(seg_img)
        all_unique = get_all_unique(all_points)
        points = get_turning_points(seg_img,tolerance=2,min_angle=np.pi*0.15)
        vessel_metrics = []
        for j in range(len(all_unique)):
            tort,dm,ip,tp,sd = tortousity(all_unique[j])
            vessel_metrics.append([tort,dm,ip,tp,sd])
            all_img_ids.append(path.split('/')[-1])
        all_metrics.append(np.array(vessel_metrics))
    df.img_id = all_img_ids
    df.iloc[:, 1:6] = np.concatenate(all_metrics)
    return df

In [3]:
img_paths1 = sorted(glob.glob('../Data/SPIE_2020_Retinal_Tortuosity/Emmetropia/*')) #Original Folder
img_paths2 = sorted(glob.glob('../Data/SPIE_2020_Retinal_Tortuosity/Myopia/*')) #Original Folder
img_paths3 = sorted(glob.glob('../Data/SPIE_2020_Retinal_Tortuosity/Hypermetropia/*')) #Original Folder
img_paths = img_paths1 + img_paths2 + img_paths3

df = calculate(img_paths)

2it [09:35, 304.31s/it]../Scripts/metrics.py:287: RuntimeWarning: invalid value encountered in double_scalars
  dm+=al/cl-1
../Scripts/metrics.py:292: RuntimeWarning: invalid value encountered in double_scalars
  dm+=al/cl-1
65it [12:08:18, 672.29s/it] 


In [35]:
img_paths1 = sorted(os.listdir('../Data/SPIE_2020_Retinal_Tortuosity/Emmetropia/')) #Original Folder
img_paths2 = sorted(os.listdir('../Data/SPIE_2020_Retinal_Tortuosity/Myopia/')) #Original Folder
img_paths3 = sorted(os.listdir('../Data/SPIE_2020_Retinal_Tortuosity/Hypermetropia/')) #Original Folder
class_mapping1 = { i : 'Emmetropia' for i in img_paths1 }
class_mapping2 = { i : 'Myopia' for i in img_paths2 }
class_mapping3 = { i : 'Hypermetropia' for i in img_paths3 }

In [36]:
class_mapping = {**class_mapping1, **class_mapping2, **class_mapping3}

In [37]:
df['Class'] = ''
for i in range(len(df)):
    df.Class.iloc[i] = class_mapping[df.img_id.iloc[i]]

/home/arnav0400/.local/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [39]:
df.Class.value_counts()

Hypermetropia    1571
Emmetropia       1365
Myopia           1364
Name: Class, dtype: int64

In [40]:
df.to_csv('../Data/complete_data.csv', index = False)